In [11]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import CRUD scripts
from aac_crud2 import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "Password1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

# Valid Rescue Breeds
preferred_breeds = [
    'Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland', # Water Rescue
    'German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky','Rottweiler', # Mountain or Wilderness Rescue
    'Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler' # Disaster Rescue or Individual Tracking
]

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([

    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Center of top
    html.Center([
        # logo with url redirect
        html.A(
            href="https://www.snhu.edu",
            children=html.Img(
                src='/assets/GraziosoSalvareLogo.png',
                style={'height': '200px'}
            ),
            target='_blank', # Opens in new tab
        ),
        # title
        html.B(html.H1('SNHU CS-340 Dashboard'))
    ]),
    html.Hr(),
    
    # Create the dropdown for filters
    html.Div([
        dcc.Dropdown(
            id='rescue-type-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'}
            ],
            placeholder="Select a Rescue Type",
            multi=False
        ),
        html.Button('Reset Filters', id='reset-button', n_clicks=0), # reset button
    ]),
    
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable='single', # restricts row selection to '1'
        sort_action='native', # Allows users to sort the list by column
        filter_action='native', # Allows the user to filter the columns
        page_size=10, # restricts the page to 10 documents per page
        selected_rows=[0] # Set the initial selected row to the first row

    ),
    html.Br(),
    html.Hr(),
    
    # Displays the map and chart side by side
    html.Div([
        html.Div(id='map-id', className='col s12 m6', style={'flex': '50%'}), # Display map
        # Display Chart with title
        html.Div([
            html.H3('Rescue Dog Outcomes', style={'textAlign': 'center'}),
            html.Div(id='chart-id')
        ], className='col s12 m6', style={'flex': '50%'})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    # Unique Identifier
    html.H6("Jesse Peterson SNHU CS-340 MongoDB Authentication")
])

#############################################
# Interaction Between Components / Controller
#############################################

# Resets the dropdown and the reset button
@app.callback(
    [Output('rescue-type-dropdown', 'value'),
    Output('reset-button', 'n_clicks')],
    [Input('reset-button', 'n_clicks')]
)
def reset_dropdown(n_clicks):
    return None, 0

# updates the table depending on the dropdown selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-dropdown', 'value'),
     Input('reset-button', 'n_clicks')]
)
def update_table(selected_rescue_types, reset_n_clicks):
    
    filtered_df=df.copy()
    
    if reset_n_clicks > 0:
        return df.to_dict(orient='records')
    

    # if selection exists
    if selected_rescue_types:
        if 'Water Rescue' in selected_rescue_types:
            filtered_df = df[
                (df['breed'].isin(preferred_breeds[:3])) & # filtering by breed
                (df['sex_upon_outcome'] == 'Intact Female') & # filtering by sex
                (df['age_upon_outcome_in_weeks'].between(26, 156)) # filtering by age 
            ]
                                                   
        if 'Mountain or Wilderness Rescue' in selected_rescue_types:
            filtered_df = df[
                (df['breed'].isin(preferred_breeds[3:8])) & # filtering by breed
                (df['sex_upon_outcome'] == 'Intact Male') & # filtering by sex
                (df['age_upon_outcome_in_weeks'].between(26, 156)) # filtering by age
            ]
            
        if 'Disaster Rescue or Individual Tracking' in selected_rescue_types:
            filtered_df = df[
                (df['breed'].isin(preferred_breeds[8:])) & # filtering by breed
                (df['sex_upon_outcome'] == 'Intact Male') & # filtering by sex
                (df['age_upon_outcome_in_weeks'].between(20, 300)) # filtering by age
            ]
        
    return filtered_df.to_dict(orient='records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):


    if viewData is None or len(viewData) == 1:
        return dl.Map(style = {'width': '600px', 'height': '300px'}, center = [30.75, -97.48], zoom=10)


    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can
    # be converted to a row index here
    if index is None or len(index) == 0 or index[0] >= len(dff):
        row = 0;
    else:
        row = index[0]
        
    # set the column data to variables
    # Increases readability
    # Column 13 and 14 define the grid-coordinates for the map
    latitude = dff.iloc[row, 13]    
    longitude = dff.iloc[row, 14]
    animalBreed = dff.iloc[row, 4] # Column 4 defines the breed for the animal
    animalName = dff.iloc[row, 9] # Column 9 defines the name of the animal

#     For debugging
#     print(f"Debug: Plotting location [{dff.iloc[row, 13]}, {dff.iloc[row, 14]}]")

    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, # lowered the dimensions, was too intensive
              center=[30.75, -97.48], zoom=10, children =[ #initializing the map
                  dl.TileLayer(id="base-layer-id"),
                  # Marker with tool tip popup
                  dl.Marker(position=[latitude, longitude], #intitializing the marker
                           children=[
                               dl.Tooltip(animalBreed), # when hover
                               dl.Popup([ #when click
                                   html.H1("Animal Name"),
                                   html.P(animalName)
                               ])
                           ])
              ])
    ]

# update the chart based on selection
@app.callback(
    Output('chart-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_chart(viewData):
    dff=pd.DataFrame.from_dict(viewData)
    
    # if the breed does not exist
    if 'breed' not in dff.columns:
        return[]
    
    # refine the selection by breed
    dog_df = dff[dff['breed'].isin(preferred_breeds)]
    
    # if breeds dont exist
    if len(dog_df) == 0:
        return[]
    
    # create the pie chart
    fig = px.pie(dog_df, names='breed')
    
    return dcc.Graph(figure=fig)

app.run_server(debug=True)

Dash app running on http://127.0.0.1:10484/
